### Importing Relevant Packages

In [ ]:
import numpy as np
import pandas as pd
import scipy
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
from statsmodels.tsa.arima_model import ARIMA
import statsmodels.graphics.tsaplots as sgt
import statsmodels.tsa.stattools as sts
from statsmodels.tsa.statespace.sarimax import SARIMAX
from pmdarima.arima import auto_arima
from arch import arch_model
import seaborn as sns
import yfinance
import warnings
warnings.filterwarnings("ignore")
sns.set()

### Loading Data and Preprocessing

In [ ]:
 raw_data = yfinance.download(tickers = "^GSPC,^FTSE,^N225,^GDAXI", start = '1994-01-07', end = '2019-09-01', 
                              interval = '1d', group_by = 'ticker', auto_adjust = True, treads = True)

In [ ]:
df_comp = raw_data.copy()

In [ ]:
df_comp['spx'] = df_comp['^GSPC'].Close[:]
df_comp['dax'] = df_comp['^GDAXI'].Close[:]
df_comp['ftse'] = df_comp['^FTSE'].Close[:]
df_comp['nikkei'] = df_comp['^N225'].Close[:]

### Deleting Unwanted Columns

In [ ]:
df_comp = df_comp.iloc[1:]
del df_comp['^N225']
del df_comp['^GSPC']
del df_comp['^GDAXI']
del df_comp['^FTSE']

In [ ]:
df_comp.isnull().sum()

### Dealing with Missing Values

In [ ]:
df_comp = df_comp.asfreq('b')
df_comp = df_comp.fillna(method = 'ffill')

### Creating Returns

In [ ]:
df_comp['ret_spx'] = df_comp.spx.pct_change(1).mul(100)
df_comp['ret_ftse'] = df_comp.ftse.pct_change(1).mul(100)
df_comp['ret_dax'] = df_comp.dax.pct_change(1).mul(100)
df_comp['ret_nikkei'] = df_comp.spx.pct_change(1).mul(100)

### Normalizing

In [ ]:
df_comp['norm_ret_spx'] = df_comp.ret_spx.div(df_comp[1:]) * 100
df_comp['norm_ret_ftse'] = df_comp.ret_ftse.div(df_comp[1:]) * 100
df_comp['norm_ret_dax'] = df_comp.ret_dax.div(df_comp[1:]) * 100
df_comp['norm_ret_nikkei'] = df_comp.ret_nikkei.div(df_comp[1:]) * 100

### Splitting the Data

In [ ]:
size = int(len(df_comp) * 0.8)
df, df_test = df_comp.iloc[:size], df_comp.iloc[size:]

### Fitting a model

In [ ]:
model_ar = ARIMA (df.ftse, order = (1,0,0))
results_ar = model_ar.fit()

### Forecasting

In [ ]:
df.tail()

In [ ]:
#Creating a starting and end date for the forecasting
start_date = '2014-07-15'
end_date = '2015-01-01'
df_pred = results_ar.predict(start = start_date, end = end_date)

In [ ]:
df_pred[start_date:end_date].plot(figsize = (20, 5), color ='red')
df_test.ftse[start_date:end_date].plot(color = 'blue')
plt.title('Actuals vs Predictions', size = 24)
plt.show()

### Forecasting for Returns

In [ ]:
end_date = '2015-01-01'
model_ret_ar = ARIMA(df.ret_ftse[1:], order = (1,0,0))
results_ret_ar = model_ret_ar.fit()

In [ ]:
df_pred_ar = results_ret_ar.predict(start = start_date, end = end_date)
df_pred_ar[start_date:end_date].plot(color = 'blue', figsize = (20,5))
df_test.ret_ftse[start_date:end_date].plot(color = 'red')
plt.title('Predictiongs vs Actual (Returns)', size = 24)
plt.show()